In [2]:
#Preflop analyzer


In [3]:
#Goal create a Faise First In(RFI) table with position 
 #howto - need to parse my hands, then determine what position im in, and determine if im the RFI
 #howto - then, i need to matplotlib 6 or so plots from the individual positions
         #- i would like this to be in the standard (A-2 * A-2) plot coloring the played range 

#Goal 2 - take the data and report some sort of percentage of play for hand per position, otherwise we may misrepresent a super large range of hands 
 #howto - we need a boolean average of hand per position reported in our figure 
#Goal 

In [4]:
import pandas as pd

In [5]:
#ignitionHandHistoryFile = input("Text file dir:")
handhistoryfilename = input("file dir:")
handhistoryfilename.replace("\\","\\\\")
                            
ignitionHands = open(handhistoryfilename,"r").read()
print(ignitionHands.split("Ignition Hand")[2])



file dir:C:\Users\edavid\Documents\Data\IPHH\1.txt
 #3915240980 TBL#21035758 HOLDEM No Limit - 2020-04-04 15:56:30
Seat 1: UTG+2 ($3.62 in chips)
Seat 2: UTG+3 ($46.94 in chips)
Seat 3: UTG+4 ($10 in chips)
Seat 4: Dealer ($1.74 in chips)
Seat 5: Small Blind [ME] ($9.90 in chips)
Seat 6: Big Blind ($10 in chips)
Seat 8: UTG ($3.19 in chips)
Seat 9: UTG+1 ($11.54 in chips)
Dealer : Set dealer [4] 
Small Blind  [ME] : Small Blind $0.05 
Big Blind : Big blind $0.10 
*** HOLE CARDS ***
UTG+2 : Card dealt to a spot [Kd 2s] 
UTG+3 : Card dealt to a spot [Js 6d] 
UTG+4 : Card dealt to a spot [Tc Ts] 
Dealer : Card dealt to a spot [Jd Qd] 
Small Blind  [ME] : Card dealt to a spot [4s 4h] 
Big Blind : Card dealt to a spot [3d 4c] 
UTG : Card dealt to a spot [Jh 6c] 
UTG+1 : Card dealt to a spot [3s 9h] 
UTG : Folds
UTG+1 : Folds
UTG+2 : Folds
UTG+3 : Folds
UTG+4 : Raises $0.50 to $0.50
Dealer : Calls $0.50 
Seat re-join
Small Blind  [ME] : Folds
Big Blind : Folds
*** FLOP *** [8s 7c 8h]
UTG+4 :

In [22]:
i=2
s = ignitionHands.split("Ignition Hand")[i]
startstringmarker = "*** HOLE CARDS ***"
endingstringmarker = "*** FLOP ***"

start = s.find(startstringmarker) + len(startstringmarker)
end = s.find(endingstringmarker)

substring = s[start:end]

substringline = substring.splitlines()[2]
if s.find(":") == -1:
    pass
else:
    positionInfo = substringline[:substringline.find(":")-1]
    substringlinewMeRemoved = substringline.replace("[Me]","")
    leftcard = substringlinewMeRemoved[substringlinewMeRemoved.find("[")+1:substringlinewMeRemoved.find("[")+3]
    rightcard = substringlinewMeRemoved[substringlinewMeRemoved.find("[")+4:substringlinewMeRemoved.find("[")+6]
    pass#extract data

for i in range(len(substringline)):
    if positionInfo in substringline:
        if 
        

print(positionInfo)
print(leftcard)
print(rightcard)

UTG+3
Js
6d


In [7]:

#Process Data
#----------Text parcing
##---------Table of hand and action 
###--------Table of RFI position 

In [2]:
def parseHandPos(preflopLine):
    #accept 1 line of preflop text 
    #returns hand and Position info
    
    if preflopBlock.find(":") == -1: #this would mean this isnt a card assignemnt line #mistakes 
        positionInfo,leftcard,rightcard = None,None,None
    else: #text parse out 
        positionInfo = substringline[:substringline.find(":")-1]
        substringlinewMeRemoved = substringline.replace("[Me]","")
        leftcard = substringlinewMeRemoved[substringlinewMeRemoved.find("[")+1:substringlinewMeRemoved.find("[")+3]
        rightcard = substringlinewMeRemoved[substringlinewMeRemoved.find("[")+4:substringlinewMeRemoved.find("[")+6]
        pass#extract data
    
    return positionInfo,leftcard,rightcard #cards have no fucntional order

In [31]:
#find RFI
def getRFIposition(preflopBlock):
    
    #assume that the first line that says the word raise should be the RFI
    # check by seing if the raise to x with total of x
    
    preflopLines = preflopBlock.splitlines()
    
    positionString= "notfound"
    for i in range(len(preflopLines)):
        textline = preflopLines[i]
        if "Raise" in textline:
            positionString = textline[:textline.find(":")-1]
    
    return positionString

In [32]:
print(getRFIposition(substring))

UTG+4


In [ ]:
#find RFI or fold 
#runs until there is a RFI, 
def raiseFirstIn(preflopblock):
    preflopLines = preflopblock.splitlines()
    
    for i in range(len(preflopLines)):
        
        
        
    
    
    
    return foldLimpRFIhands

#preflop block 


#return dataframe to append 

In [8]:
preflopdata = pd.DataFrame(columns = ["Hand Index","Table Position","Hand"])


In [9]:
preflopdata.head()

,Hand Index,Table Position,Hand
